In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [5]:
path = '../JasonMHoughton/Documents/Classwork/GWU MSBA/SUMMER 2017/Machine Learning/'
train_file = (path+"train.csv")
test_file = (path+"test.csv")
output_file = (path+"submission.csv")

In [6]:
raw_data = np.loadtxt(train_file, skiprows=1, dtype='int', delimiter=',')
x_train, x_val, y_train, y_val = train_test_split(
    raw_data[:,1:], raw_data[:,0], test_size=0.1)

In [8]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)

In [9]:
x_train = x_train.astype("float32")/255.
x_val = x_val.astype("float32")/255.

In [10]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]


In [12]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',
                 input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [13]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [14]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [15]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [16]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=500,
                           epochs=20, 
                           verbose=2,  #1 for ETA, 0 for silent
                           validation_data=(x_val[:400,:], y_val[:400,:]), #For speed
                           callbacks=[annealer])

Epoch 1/20
31s - loss: 0.8877 - acc: 0.7210 - val_loss: 0.6071 - val_acc: 0.8350
Epoch 2/20
30s - loss: 0.3715 - acc: 0.8846 - val_loss: 0.0746 - val_acc: 0.9750
Epoch 3/20
30s - loss: 0.2904 - acc: 0.9146 - val_loss: 0.0778 - val_acc: 0.9825
Epoch 4/20
30s - loss: 0.2468 - acc: 0.9289 - val_loss: 0.0544 - val_acc: 0.9850
Epoch 5/20
37s - loss: 0.2150 - acc: 0.9381 - val_loss: 0.0513 - val_acc: 0.9825
Epoch 6/20
32s - loss: 0.1902 - acc: 0.9445 - val_loss: 0.0355 - val_acc: 0.9875
Epoch 7/20
32s - loss: 0.1710 - acc: 0.9533 - val_loss: 0.0398 - val_acc: 0.9875
Epoch 8/20
33s - loss: 0.1599 - acc: 0.9544 - val_loss: 0.0212 - val_acc: 0.9975
Epoch 9/20
35s - loss: 0.1558 - acc: 0.9552 - val_loss: 0.0327 - val_acc: 0.9850
Epoch 10/20
38s - loss: 0.1392 - acc: 0.9611 - val_loss: 0.0163 - val_acc: 0.9950
Epoch 11/20
31s - loss: 0.1261 - acc: 0.9637 - val_loss: 0.0241 - val_acc: 0.9900
Epoch 12/20
30s - loss: 0.1126 - acc: 0.9659 - val_loss: 0.0456 - val_acc: 0.9900
Epoch 13/20
30s - loss: 0

In [17]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

Final loss: 0.0302, final accuracy: 0.9914


In [19]:
y_hat = model.predict(x_val)
y_pred = np.argmax(y_hat, axis=1)
y_true = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[438   1   0   0   0   0   2   0   0   0]
 [  0 496   0   0   0   0   0   2   0   0]
 [  1   0 388   0   0   0   0   1   1   0]
 [  1   0   0 408   0   0   0   1   3   1]
 [  0   2   0   0 426   0   0   0   0   3]
 [  0   0   0   0   0 396   3   0   0   0]
 [  0   0   0   0   0   0 403   0   0   0]
 [  0   1   0   0   0   0   0 438   0   0]
 [  0   2   0   0   1   1   2   0 386   1]
 [  2   1   0   0   1   0   0   1   1 385]]


In [20]:
mnist_testset = np.loadtxt(test_file, skiprows=1, dtype='int', delimiter=',')
x_test = mnist_testset.astype("float32")
x_test = x_test.reshape(-1, 28, 28, 1)/255.

In [21]:
y_hat = model.predict(x_test, batch_size=64)

In [22]:
y_pred = np.argmax(y_hat,axis=1)

In [23]:
with open(output_file, 'w') as f :
    f.write('ImageId,Label\n')
    for i in range(len(y_pred)) :
        f.write("".join([str(i+1),',',str(y_pred[i]),'\n']))